## **Quick and easy use of KA-Search**

For an easy search without much configuration, use the EasySearch module. This module only allows you to run a single query at a time on a single region. For running more queries and searching for more regions simultaneously, look at using KA-Search with more configuration.

In [1]:
from kasearch import EasySearch

In [2]:
query = 'QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSS'

results = EasySearch(query, 
               keep_best_n=5,
               database_path='oasdb-small', 
               allowed_chain='Any', 
               allowed_species='Any',
               regions=['whole'],
               length_matched=[False],
               n_jobs=10,
              )

results

,sequence,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call,d_call,...,Longitudinal,Vaccine,Age,Disease,Subject,Chain,Unique sequences,Isotype,Total sequences,Identity
0,TCAGCCCTCCTTTAATTCCCCAGGTTCAACTGCAGCAATCTGGGGC...,H,F,T,F,T,F,T,IGHV1-87*01,IGHD2-1*01,...,no,None,no,None,Balb/c-Mouse-3,Heavy,4486242,IGHM,6996230,0.907563
1,CATTCCCTCCTTTAATTCCCCAGGTTCAGCTGCAGCAGTCTGGGGC...,H,F,T,F,T,F,T,IGHV1-87*01,NaN,...,no,None,no,None,Balb/c-Mouse-3,Heavy,4486242,IGHM,6996230,0.907563
2,GATGCCCTCCTTTAATTCCCCAGGTTCAGCTGCAGCAGTCTGGGGC...,H,F,T,F,T,T,T,IGHV1-87*01,IGHD2-1*01,...,no,None,no,None,Balb/c-Mouse-1,Heavy,4485747,IGHM,8694330,0.907563
3,AGGGCCCTCCTTTAATTCCCCAGGTTCAGCTGCAGCAGTCTGGGGC...,H,F,T,F,T,T,T,IGHV1-87*01,IGHD2-1*01,...,no,None,no,None,Balb/c-Mouse-1,Heavy,4485747,IGHM,8694330,0.907563
4,TTGTCCCTCCTTTAATTCCCCAGGTTCAGCTGCAGCAGTCTGGGGC...,H,F,T,F,T,T,T,IGHV1-87*01,IGHD2-3*01,...,no,None,no,None,Balb/c-Mouse-2,Heavy,4220572,IGHM,6206190,0.907563


------------------
## **KA-Search with more configuration**

In [3]:
from kasearch import AlignSequences, SearchDB, PrepareDB

### **Align query sequences**

Sequences to search with needs to be aligned to the KA-Search alignment as described in Olsen et al, 2022.

In [4]:
raw_queries = [
    'QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSS',
    'QVQLKESGPGLVAPSQSLSITCTVSGFSLTSYGVSWVRQPPGKGLEWLGVIWGDGSTNYHSALISRLSISKENSKSQVFLKLNSLQTDDTATYYCAKPGGDYWGQGTSVTVSS',
]

aligned_seqs = AlignSequences(allowed_species=['Human', 'Mouse'], # Species to use for numbering (human and mouse is default).
                              n_jobs=1                            # Allocated number for jobs/threads for the search.
                             )(raw_queries)                       # Sequences as strings to align.
aligned_seqs

array([[81, 86, 75,  0, 76, 81, 69, 83, 71, 65,  0, 69, 76, 65, 82, 80,
        71, 65, 83, 86, 75, 76, 83, 67, 75, 65, 83, 71, 89, 84, 70,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 84, 78, 89, 87, 77, 81,  0, 87,
        86, 75, 81,  0, 82,  0, 80,  0, 71,  0, 81,  0,  0, 71,  0, 76,
        68,  0, 87, 73, 71, 65, 73, 89, 80, 71,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0, 68, 71, 78, 84, 82, 89,  0,  0, 84,  0,  0,
        72,  0,  0, 75, 70,  0,  0, 75,  0,  0,  0, 71, 75, 65, 84, 76,
        84, 65,  0, 68,  0,  0,  0, 75,  0, 83,  0,  0, 83, 83,  0,  0,
         0,  0, 84,  0, 65, 89, 77, 81, 76, 83, 83, 76, 65, 83,  0, 69,
        68, 83, 71, 86, 89, 89, 67, 65, 82, 71, 69, 71, 78,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, 89, 65, 87, 70, 65, 89, 87, 71,  0, 81,
        71, 84, 84, 86, 84, 86, 83, 83],
       [81, 86, 81,  0, 76, 75, 69, 83, 71, 80,  0, 71, 76, 86, 65, 80,
        83, 81, 83, 76,

--------------
### **Initiate search class**

#### Database to search against
- If no database path is given, a small OAS version will be downloaded to search against.
- The full version of OAS can be downloaded here ().
- You can also give it the path for a custom database to search against. (See below for how to create a custom database).
- You can place the custom database in the OAS folder to have KA-Search search against both databases.

#### Regions to search with
- Default regions are the whole chain, CDRs or CDR3.
- User-defined regions can be added, as seen with the paratope search below.
- For each region, the search can either be based on exact length match or not.
- For a more specific search, the search can be focused on a specific chain and species.

In [5]:
paratope = ["107 ", "108 ","111C", "114 ","115 "]

In [6]:
searchdb = SearchDB(
    database_path='oasdb-small',   # Path to your database. Default will be to download a small prepared version of OAS.
    allowed_chain='Heavy',         # Search against a specific chain. Default is any chain.
    allowed_species='Human',       # Search against a specific species. Default is any species.
    regions=['whole', 'cdrs', 'cdr3', paratope], # Regions to search with.
    length_matched=[False, True, True, True],    # Whether to search with length match or not.
)

-----------
### **Run search**

A search takes ~23min per sequence against all of OAS and ~2min per sequence against the small OAS.

To specify the number of closest similar sequences to keep, you can change the keep_best_n parameter.

In [7]:
%%time 
searchdb.search(aligned_seqs,   # Input can be a single or multiple aligned sequences at a time.
                keep_best_n=5,  # You can define how many most similar sequences to return
            )

CPU times: user 6min 22s, sys: 5min 22s, total: 11min 45s
Wall time: 3min 44s


### Get N best identities

Identities of the most similar sequence for each of the regions can be fetched from the object with the bellow command.

In [8]:
searchdb.current_best_identities

array([[[0.90378153, 0.9461583 , 0.78571427, 0.8636364 ],
        [0.8983696 , 0.94224083, 0.75      , 0.77272725],
        [0.89311516, 0.9380531 , 0.75      , 0.77272725],
        [0.8925946 , 0.93393886, 0.75      , 0.72727275],
        [0.8907563 , 0.9286455 , 0.71428573, 0.72727275]],

       [[0.8333333 , 1.        , 1.        , 1.        ],
        [0.8333333 , 1.        , 1.        , 1.        ],
        [0.8333333 , 1.        , 1.        , 1.        ],
        [0.8333333 , 1.        , 1.        , 1.        ],
        [0.8333333 , 0.85714287, 1.        , 1.        ]]], dtype=float32)

---------
## Extract the meta data from matched sequences

Using the get_meta function, the meta data for all matched sequences for each query and region can be extracted as seen below.

Zero (0) is the first query or the first region in the list when initiating the search class. 

NB: The column "sequence_alignment_aa" holds the antibody sequence.

In [9]:
n_best_sequences = searchdb.get_meta(n_query = 0,          # Which query to extract meta data from
                                     n_region = 0,         # Which region to extract meta data from
                                     n_sequences = 'all',  # Number of sequences to extract (default is all, which is keep_best_n)
                                     n_jobs=10             # Allocated number for jobs/threads for the extraction
                                    )
n_best_sequences

,sequence,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call,d_call,...,Longitudinal,Age,Disease,Subject,Vaccine,Chain,Unique sequences,Total sequences,Isotype,Identity
0,GAAACAACCTATGATCAGTGTCCTCTCTACACAGTCCCTGACGACA...,H,F,T,F,T,F,F,IGHV1-46*01,IGHD3-16*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.903782
1,GGCATATGATCAGTAACCTCTTCACAGTCACTGAAAACACTGACTC...,H,F,T,F,T,F,F,IGHV1-46*01,IGHD5-12*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.898370
2,GGCATATGATCAGTAACCTCTTCACAGTCACTGAAAACACTGACTC...,H,F,T,F,T,F,F,IGHV1-46*01,IGHD3-9*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.893115
3,GACAGTCACTGAAAACACTGACTCTAATCATGGAATGTAACTGGAT...,H,F,T,F,T,F,T,IGHV1-46*01,IGHD5-12*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.892595
4,GGCATATGATCAGTAACCTCTTCACAGTCACTGAAAACACTGACTC...,H,F,T,F,T,F,F,IGHV1-46*01,IGHD3-3*01,...,no,no,POEMS,Patient_12,None,Heavy,21060,37905,Bulk,0.890756


In [10]:
n_best_sequences.sequence_alignment_aa.values

array(['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGEPRYDYAWFAYWGQGTLVTVS',
       'QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGPATAWFAYWGQGTLVTVS',
       'QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARSAWFAYWGQGTLVTVS',
       'QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGGYWGQGTTLTVSS',
       'QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGGLRRGAWFAYWGQGTLVTVS'],
      dtype=object)

----------
## Create custom database


To create your own database you first need to create a csv file in the OAS format. For an example file, look at data/custom-data-example.csv. This file consists of a dictionary containing the metadata in the first line and then rows of the individual sequences afterwards. Only the Species and Chain is strictly needed in the metadata, and only the amino acids sequence of the antibodies is required for each antibody sequence.

### 1. Format your data into OAS files

In [11]:
import json
import pandas as pd

In [12]:
metadata = {"Species":"Human", "Chain":"Heavy"}
metadata = pd.Series(name=json.dumps(metadata), dtype='object')
seqsdata = pd.DataFrame([["EVQLVESGGGLAKPGGSLRLHCAASGFAFSSYWMNWVRQAPGKRLEWVSAINLGGGLTYYAASVKGRFTISRDNSKNTLSLQMNSLRAEDTAVYYCATDYCSSTYCSPVGDYWGQGVLVTVSS"],
                         ["EVQLVQSGAEVKRPGESLKISCKTSGYSFTSYWISWVRQMPGKGLEWMGAIDPSDSDTRYNPSFQGQVTISADKSISTAYLQWSRLKASDTATYYCAIKKYCTGSGCRRWYFDLWGPGT"],
                         ['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGEPRYDYAWFAYWGQGTLVTVS'],
                         ['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGPATAWFAYWGQGTLVTVS'],
                         ['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARSAWFAYWGQGTLVTVS'],
                         ['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGGYWGQGTTLTVSS'],
                         ['QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLEWIGAIYPGDGDTRYTQKFKGKATLTADKSSSTAYMQLSSLASEDSAVYYCARGGLRRGAWFAYWGQGTLVTVS']
                         ], columns = ['sequence_alignment_aa'])

In [13]:
save_file = "../data/custom-data-examples.csv"
metadata.to_csv(save_file, index=False)
seqsdata.to_csv(save_file, index=False, mode='a')

### 2. Turn your OAS formatted files into a custom database

After creating all the files you want to include in the new database, you can run the following code to create the database.

In [14]:
kasearch_db_path = "../data/my_kasearch_db"
data_files_to_prepare = ['../data/custom-data-examples.csv']

In [15]:
%%timeit -n 1 -r 1
newDB = PrepareDB(db_path=kasearch_db_path, n_jobs=20)

for num, data_unit_file in enumerate(data_files_to_prepare):

    metadata = json.loads(','.join(pd.read_csv(data_unit_file, nrows=0).columns))
    newDB.prepare_sequences(data_unit_file,
                            file_id=num, 
                            chain=metadata['Chain'], 
                            species=metadata['Species'])
    
newDB.save_database()
newDB.merge_sequence_files()

592 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 3. Initiate the search class with your custom database


In [16]:
query = 'QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSS'

results = EasySearch(query, 
               keep_best_n=5,
               database_path=kasearch_db_path, 
               allowed_chain='Any', 
               allowed_species='Any',
               regions=['whole'],
               length_matched=[False],
               n_jobs=10,
              )

results

,sequence_alignment_aa,Species,Chain,Identity
0,QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLE...,Human,Heavy,0.903782
1,QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLE...,Human,Heavy,0.903782
2,QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLE...,Human,Heavy,0.903782
3,QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLE...,Human,Heavy,0.903782
4,QVQLQQSGAELARPGASVKLSCKASGYTFTSYWMQWVKQRPGQGLE...,Human,Heavy,0.903782
